# HMM

* ** HMM Formalism ($HMM\space\lambda = (A,B)$) ** (cf. J&M ch5.5:23)

    * $Q = q_1...q_N$, a set of $N$ *states*.
    * $A = a_{11}a_{12}...a_{nn}$, where $\sum_{j=1}^na_{ij} = 1, \forall i$, a *transition probability matrix* $A$.
    * $O = o_1...o_T$, a sequence of *observations*.
    * $B = b_i(o_t)$, a sequence of *emission probabilities* (i.e. the probability of an observation $o_t$ being omitted at state $i$.
    * $q_0,q_F$, *start state* and *end state*.


* ** Framing Problems ** (cf. J&M ch6.2:7)

    * ** Problem 1 (Computing Likelihood): ** Given an HMM $\lambda=(A,B)$ and an observation sequence $O$, determine the likelihood $P(O|\lambda)$.

    * ** Problem 2 (Decoding): ** Given an observation sequence $O$ and an HMM $\lambda=(A,B)$, discover the best hidden state sequence $Q$.

    * ** Problem 3 (Learning): ** Given an observation sequence $O$ and the set of states in the HMM, learn the HMM parameters $A$ and $B$. 

### A. Computing Likelihood

**NB: The following version of forward pass is non-generalizable, written out in syntactically transparent version for demonstration purpose. A serious version will come up in section C. **

In [ ]:
# ICE CREAM MODEL
 
#  -.8-> HOT -.7-> HOT -.3-> COLD
#         |         |         |
#        .4        .2        .1
#         3         1         3

# Question: P(O = [3,1,3])

* ** Math ** (cf. J&M ch6.3:9,eq.6.10,12)

    * $P(O,Q) = P(O|Q)\cdot P(Q) = \prod_{i=1}^nP(o_i|q_i)\times\prod_{i=1}^nP(q_i|q_{i-1})$, where $n$ is the ordered index sequence of observations.
    
    * $P(O) = \sum_QP(O,Q) = \sum_QP(O|Q)\cdot P(Q)$
    

* ** Computation ** (cf. ibid.:10-12)

    * Brute-Force: $O(N^T)$, where $N$ is the set of tags, $T$ the length of observation sequence.
    
    * Forward Algorithm: $O(N^2T)$.

In [267]:
# TOY HMM
class LMD:
    def __init__(self):
        self.Q = ['START','COLD','HOT'] # NB: the order matters for indexing reason later.
        self.A = {('START','START'):0.,('START','HOT'):.8,('START','COLD'):.2,
                  ('HOT','START'):0.,('HOT','HOT'):.7,('HOT','COLD'):.3,
                  ('COLD','START'):0.,('COLD','HOT'):.4,('COLD','COLD'):.6}
        self.B = {('HOT',1):.2,('HOT',2):.4,('HOT',3):.4,
                  ('COLD',1):.5,('COLD',2):.4,('COLD',3):.1}

##### Brute-Force "Baseline"

* num_states | processing time:
    * 3 | 300 $\mu$s
    * 15 | 800 ms
    * 20 | 30 s

In [268]:
from itertools import product
from nltk.util import ngrams
from operator import mul

In [303]:
def brute_force(lmd,O,num_states=3):
    Q, A, B = lmd.Q, lmd.A, lmd.B
    Qs = [list(Q) for Q in product(Q[1:],repeat=num_states)] # all possible t-state sequences.
    # computation components
    def Pr_Q(Q): # Q = [q_1,...,q_n].
        Q = deepcopy(Q)
        Q.insert(0,'START')
        p = 1.
        for q_trans in ngrams(Q,2): # q_trans = (q_i-1,q_i).
            p *= A[q_trans]
        return p
    def Pr_O_given_Q(O,Q): # O = [o_1,...,o_t], Q = [q_1,...,q_n].
        return reduce(mul,map(B.get,zip(Q,O)))
    def Pr_O(O):
        return sum(Pr_O_given_Q(O,Q)*Pr_Q(Q) for Q in Qs)      
    print Pr_O(O)

In [315]:
%%time
lmd = LMD()
O = [3,1,3]
brute_force(lmd,O,num_states=20)

0.026264
CPU times: user 31.8 s, sys: 230 ms, total: 32 s
Wall time: 32 s


##### Forward Algorithm

* num_states | processing time:
    * 3 | 300 $\mu$s
    * 15 | 400 $\mu$s
    * 20 | 500 $\mu$s

In [294]:
import numpy as np

In [320]:
def forward(lmd,O):
    Q, A, B = lmd.Q, lmd.A, lmd.B
    N, T = len(Q), len(O)
    fwd = np.zeros((N,T))
    for s in xrange(1,N):
        fwd[s][1] = A[('START',Q[s])] * B[(Q[s],O[1])]
            # initialize: START -> all states
    for t in xrange(2,T):
        for s in xrange(1,N):
            fwd[s][t] = sum(fwd[s_prime][t-1]*A[(Q[s_prime],Q[s])]*B[(Q[s],O[t])]
                            for s_prime in xrange(1,N)) 
            # forward passing: sum ( each prev state * prev->current * current->omission )
    print 'Pr(O) = ', sum(fwd[:,-1])
    
# fwd matrix in when O = [3,1,3], (cf. J&M ch6.4:10,fig.6.7)
# [[ 0.        0.        0.        0.      ]
#  [ 0.        0.02      0.054     0.004632]
#  [ 0.        0.32      0.0464    0.021632]]

In [321]:
%%time
O = ['<s>',3,1,3]
forward(LMD(),O)

Pr(O) =  0.026264
CPU times: user 271 µs, sys: 94 µs, total: 365 µs
Wall time: 293 µs


### B. Decoding: Viterbi

In [318]:
# TOY HMM again
class LMD:
    def __init__(self):
        self.Q = ['START','COLD','HOT'] # NB: the order matters for indexing reason later.
        self.A = {('START','START'):0.,('START','HOT'):.8,('START','COLD'):.2,
                  ('HOT','START'):0.,('HOT','HOT'):.7,('HOT','COLD'):.3,
                  ('COLD','START'):0.,('COLD','HOT'):.4,('COLD','COLD'):.6}
        self.B = {('HOT',1):.2,('HOT',2):.4,('HOT',3):.4,
                  ('COLD',1):.5,('COLD',2):.4,('COLD',3):.1}

In [319]:
import numpy as np

* ** Math (Recursion Step) **

    * $viterbi_{s,t} = max(viterbi_{s',t-1} * A_{s',s} * B_{s,t})$, i.e. at each time $t$, at state $s$, set viterbi as **the largest value** of the multiplication $\mathtt{value\,of\,prev\,state}\times\mathtt{transition\,from\,prev\,to\,current}\times\mathtt{current\,emission}$.
    
    * $backpointer_{s,t} = argmax(viterbi_{s',t-1} * A_{s',s})$, i.e. at each time $t$, at state $s$, set backpointer as the **index of largest value** of the multiplication $\mathtt{value\,of\,prev\,state}\times\mathtt{transition\,from\,prev\,to\,current}$.

In [449]:
def viterbi(lmd,O):
    Q, A, B = lmd.Q, lmd.A, lmd.B
    N, T = len(Q), len(O)
    viterbi, backpointer = np.zeros((N,T)), np.zeros((N,T),dtype=int)
    for s in xrange(1,N): 
        # this is written in a more transparent syntax, for simplification, see J&M_02.
        viterbi[s][1] = A[('START',Q[s])] * B[(Q[s],O[1])]
        backpointer[s][1] = 0
    for t in xrange(2,T):
        for s in xrange(1,N):
            viterbi[s][t] = max(viterbi[s_prime][t-1]*A[(Q[s_prime],Q[s])]*B[(Q[s],O[t])]
                                for s_prime in xrange(1,N))
            backpointer[s][t] = np.argmax([viterbi[s_prime][t-1]*A[(Q[s_prime],Q[s])]
                                           for s_prime in xrange(1,N)])+1
    max_state = np.argmax(viterbi[:,-1])
    best_state_seq = []
    for t in reversed(xrange(0,T)): 
        best_state_seq.insert(0,Q[max_state])
        max_state = backpointer[max_state][t]
    
    print 'viterbi'
    print viterbi
    print 'backpointer'
    print backpointer
    print best_state_seq


In [450]:
%%time
O = ['<s>',3,1,3]
viterbi(LMD(),O) 
    # NB: J&M ch6.4:13,fig.6.10 is not showing the best tag sequence for O=[3,1,3],
    #  specifically, the figure is inconsistent with the transition/emission probabilities
    #  the example is working with (e.g. the numbers in LMD).

viterbi
[[ 0.        0.        0.        0.      ]
 [ 0.        0.02      0.048     0.00288 ]
 [ 0.        0.32      0.0448    0.012544]]
backpointer
[[0 0 0 0]
 [0 0 2 1]
 [0 0 2 2]]
['START', 'HOT', 'HOT', 'HOT']
CPU times: user 1.48 ms, sys: 531 µs, total: 2.01 ms
Wall time: 1.58 ms


##### Trigram HMM + Deleted Interpolation (Brown Corpus)

**Comments: Somewhat disapointing, it doesn't outperform the previous bigram HMM.**

In [682]:
def load_brown(train_percentage=.8): # importing tagged brown.
    from nltk.corpus import brown
    from nltk.stem import PorterStemmer
    print "... loading sentences"
    brown_tagged_sents = brown.tagged_sents(tagset='universal')
    print "... stemming and lowercasing words"
    brown_tagged_sents = [[(PorterStemmer().stem(w).lower(),t) for w,t in tagged_sent]
                   for tagged_sent in brown_tagged_sents]
    print "... padding sentences"
    brown_tagged_sents = [[('<s>','START')]+s+[('</s>','END')] for s in brown_tagged_sents]
    cut_off = int(len(brown_tagged_sents)*.8)
    
    return (brown_tagged_sents[:cut_off], brown_tagged_sents[cut_off:]) # train,test: two lists of sents.

In [683]:
%%time
brown_tagged_train, brown_tagged_test = load_brown()

... loading sentences
... stemming and lowercasing words
... padding sentences
CPU times: user 24.1 s, sys: 500 ms, total: 24.6 s
Wall time: 24.9 s


In [493]:
import random
import numpy as np
from nltk.util import ngrams
from __future__ import division
from collections import Counter
from numpy import isnan

In [497]:
class HMM:
    
    def __init__(self, train, test):
        self.train = train
        self.test = test
        print "... setting up vocabularies for tags and words"
        self.size = sum(map(len,self.train))
        self.vocab = list({w for sent in self.train for w,t in sent}) + ['unk'] # brown: len=31661, as T.
        self.tagset = list({t for sent in self.train for w,t in sent}) + ['UNK'] # brown: len=452, as N.
        self.btagset = Counter(ngrams([t for sent in self.train for w,t in sent],2)).keys()
        self.w2i = Counter({w:i for i,w in enumerate(self.vocab)})
        self.t2i = Counter({t:i for i,t in enumerate(self.tagset)})
        self.bt2i = Counter({bt:i for i,bt in enumerate(self.btagset)})
        print "... building transition probability matrix"
        self.build_transition_matrix()
        print "... building emission probability matrix"
        self.build_emission_matrix()
    
    def build_transition_matrix(self):
        # Comments: The following can be improved my cramping all
        #  three ngram matrices into one, however, I suspect the 
        #  gigantic matrix resulted from that will cause some
        #  computational difficulties.
        
        print "    ... computing unigram probabilities"
        unigram_dict = Counter(t for sent in self.train for w,t in sent)
        self.A_uni = np.ones(len(self.tagset))
        for t in self.tagset:
            count = unigram_dict[t]
            self.A_uni[self.t2i[t]] = count if count!=0 else 1.
        row_sum = sum(self.A_uni)
        self.A_uni /= row_sum
        self.A_uni[isnan(self.A_uni)] = 0.
        
        print "    ... computing bigram transition probabilities"
        bi_trans_dict = Counter(ngrams([t for sent in self.train for w,t in sent],2))
        self.A_bi = np.ones((len(self.tagset),len(self.tagset))) 
        for from_t in self.tagset:
            for to_t in self.tagset:
                trans_count = bi_trans_dict[(from_t,to_t)]
                self.A_bi[self.t2i[from_t]][self.t2i[to_t]] = trans_count if trans_count!=0 else 1.
        row_sums = np.apply_along_axis(sum, 1, self.A_bi)[:,np.newaxis]
        self.A_bi /= row_sums
        self.A_bi[isnan(self.A_bi)] = 0.
        
        print "    ... computing trigram transition probabilities"
        tri_trans_dict = Counter(ngrams([t for sent in self.train for w,t in sent],3))
        self.A_tri = np.ones((len(self.btagset),len(self.tagset)))
        for from_t2,from_t1 in self.btagset: # i.e. t_{i-2}, t_{i-1}
            for to_t in self.tagset:
                trans_count = tri_trans_dict[(from_t2,from_t1,to_t)]
                self.A_tri[self.bt2i[(from_t2,from_t1)]][self.t2i[to_t]] = trans_count if trans_count!=0 else 1.
        row_sums = np.apply_along_axis(sum, 1, self.A_tri)[:,np.newaxis]
        self.A_tri /= row_sums
        self.A_tri[isnan(self.A_tri)] = 0.
        
        print "    ... computing deleted interpolation coefficients"
        self.lmd1, self.lmd2, self.lmd3 = self.deleted_interpolation(unigram_dict,bi_trans_dict,tri_trans_dict)
    
    def build_emission_matrix(self):
        print "    ... counting emissions"
        emission_dict = Counter((w,t) for sent in self.train for w,t in sent)
        print "    ... computing emission probabilities"
        self.B = np.ones((len(self.tagset),len(self.vocab)))
        for t in self.tagset:
            for o in self.vocab:
                emit_count = emission_dict[(o,t)]
                self.B[self.t2i[t]][self.w2i[o]] = emit_count if emit_count!=0 else 1.
        row_sums = np.apply_along_axis(sum, 1, self.B)[:,np.newaxis]
        self.B /= row_sums
        self.B[isnan(self.B)] = 0.
    
    def deleted_interpolation(self, unigrams, bigrams, trigrams):
        # unigrams, bigrams, trigrams: 3 Counters.
        def brants_divide(num,denom): # (cf. Brants(2000):226,fig.1)
            return 0 if denom==0 else num/denom
        lmds = {1:0, 2:0, 3:0}
        for t1,t2,t3 in trigrams:
            f123 = trigrams[(t1,t2,t3)] / bigrams[(t1,t2)]
            lmds[np.argmax([-np.inf,
                            brants_divide(trigrams[(t1,t2,t3)]-1,bigrams[(t1,t2)]-1),
                            brants_divide(bigrams[(t2,t3)]-1,unigrams[t3]-1),
                            brants_divide(unigrams[t3]-1,self.size-1)])] += f123
        return np.array([lmds[1],lmds[2],lmds[3]]) / sum(np.array([lmds[1],lmds[2],lmds[3]]))

    def viterbi(self, sent):
        # sent: a sentence in the form, e.g. ['<s>',w1,...,wN,'</s>'].
        
        N, T = len(self.tagset), len(sent)
        viterbi = np.zeros((N,T))
        backpointer = np.zeros((N,T),dtype=int)
        
        for s in xrange(N):
            viterbi[s][1] = self.A_bi[self.t2i['START']][s] * self.B[s][self.w2i[sent[1]]]
            backpointer[s][1] = 0            
            
        for t in xrange(2,T):
            for s in xrange(N):
                prev_viterbi = []
                prev_backpointer = []
                for s_prime in xrange(N):
                    max_trans = max(self.lmd1*self.A_uni[s] + \
                                    self.lmd2*self.A_bi[s_prime][s] + \
                                    self.lmd3*self.A_tri[self.bt2i[(self.tagset[s_prime_prime],
                                                                    self.tagset[s_prime])]][s]
                                    for s_prime_prime in xrange(N))
                    prev_viterbi.append(viterbi[s_prime][t-1] * max_trans * self.B[s][self.w2i[sent[t]]])
                    prev_backpointer.append(viterbi[s_prime][t-1] * max_trans)
                viterbi[s][t] = max(prev_viterbi)
                backpointer[s][t] = np.argmax(prev_backpointer)  
                     
        max_state = self.tagset.index('END') 
            # TEST: always give </s> END as tag.
            #  otherwise: max_state = np.argmax(viterbi[:,T-1])
        best_tagged_seq = []
        for t in reversed(xrange(0,T)):
            best_tagged_seq.insert(0,(sent[t],self.tagset[max_state]))
            max_state = backpointer[max_state][t]
        
        return best_tagged_seq
    
    def tag(self, sent): # may use this for other passing purpose.
        return self.viterbi(sent) 

    def evaluate(self, k=500, verbose_freq=100):
        sample_idxs = random.sample(xrange(len(self.test)),k)
        sample_test = [self.test[i] for i in sample_idxs] 
        accuracies = []
        sent_lens = []
        for i in xrange(k):
            sent = [w if w in self.vocab else 'unk' for w,t in sample_test[i]]
            tagged = self.tag(sent)
            accuracy = sum(y==yhat for y,yhat in zip(sample_test[i][1:-1],tagged[1:-1])) / \
                                                                  len(tagged[1:-1])
            if i!=0 and i%verbose_freq==0:
                print "    ... tagged %d sentences, current accuracy: %.2f%%" % (i,np.mean(accuracies)*100) 
            accuracies += [accuracy]
            sent_lens += [len(tagged[1:-1])] 
            
        return [sent_lens, accuracies]
            

In [498]:
%%time
hmm = HMM(brown_tagged_train,brown_tagged_test)

... setting up vocabularies for tags and words
... building transition probability matrix
    ... computing unigram probabilities
    ... computing bigram transition probabilities
    ... computing trigram transition probabilities
    ... computing deleted interpolation coefficients
... building emission probability matrix
    ... counting emissions
    ... computing emission probabilities
CPU times: user 5.54 s, sys: 182 ms, total: 5.72 s
Wall time: 5.63 s


In [499]:
%%time
sent_lens, accuracies = hmm.evaluate(k=2000,verbose_freq=100)

    ... tagged 100 sentences, current accuracy: 90.21%
    ... tagged 200 sentences, current accuracy: 90.74%
    ... tagged 300 sentences, current accuracy: 90.28%
    ... tagged 400 sentences, current accuracy: 89.89%
    ... tagged 500 sentences, current accuracy: 90.12%
    ... tagged 600 sentences, current accuracy: 89.99%
    ... tagged 700 sentences, current accuracy: 90.16%
    ... tagged 800 sentences, current accuracy: 90.25%
    ... tagged 900 sentences, current accuracy: 90.35%
    ... tagged 1000 sentences, current accuracy: 90.16%
    ... tagged 1100 sentences, current accuracy: 90.19%
    ... tagged 1200 sentences, current accuracy: 90.22%
    ... tagged 1300 sentences, current accuracy: 90.23%
    ... tagged 1400 sentences, current accuracy: 90.28%
    ... tagged 1500 sentences, current accuracy: 90.29%
    ... tagged 1600 sentences, current accuracy: 90.24%
    ... tagged 1700 sentences, current accuracy: 90.27%
    ... tagged 1800 sentences, current accuracy: 90.26%
 

### C. Learning: Forward-Backward

**Tip:** To check forward and backward passings are implemented correctly, compute $P(O|\lambda)$ on both, the results should be the same.

In [718]:
# np.set_printoptions(formatter={'float': '{: 0.15f}'.format})
np.set_printoptions(suppress=True) # suppress pain-in-the-neck scientific-notation output in np matrices.
def prt(num): print '{:.20f}'.format(num) # print a single number in non-scientific-notation.

In [744]:
# TOY HMM
LMD = {'Q':['PPSS','VB','TO','NN'], 
       'A': np.array([[.00013, .23,     .00079,   .0012],  # rows: from-states, cols: to-states.
                      [.007,   .0038,   .035,     .047],   # rows=cols={PPSS,VB,TO,NN}, len=4
                      [0.,     .83,     0.,       .00047], # e.g. A[2][1] = p(TO->VB) = .83.
                      [.0045,  .004,    .016,     .087]]),
       'B': np.array([[.37,    0.,     0.,     0.,  ],    # rows: emitting-state, cols: emitted-observations.
                      [0.,     .0093,  0.,     .00012],   #  rows={PPSS,VB,TO,NN}, len=4
                      [0.,     0.,     .99,    0.],       #  cols={i,want,to,race}, len=4  
                      [0.,     .000054,0.,     .00057]]), # e.g. B[1][1] = p(VB->want) = .0093.
       'from_start_probs': [.067, .019, .0043, .041],
       'to_end_probs': [.001, .05, .001, .13]}

O = ['i','want','to','race']

##### Computing Forward Probability $\alpha_t(j)$ (state=j at time=t), given HMM $\lambda=\{A,B\}$ and Observation $O$.

**meaning:** The probability of ending up in state $j$ at time $t$.

In [745]:
def forward(observation=O,HMM=LMD): # (cf. J&M ch6.3:12,fig.6.9, with indexing from 0 instead)
    Q, A, B = HMM['Q'], HMM['A'], HMM['B']
    from_start = HMM['from_start_probs']
    to_end = HMM['to_end_probs']
    O = observation
    N, T = len(Q), len(O)
    fwd = np.zeros((N,T))
    for s in xrange(N):
        fwd[s][0] = from_start[s] * B[s][0]
    for t in xrange(1,T):
        for s in xrange(N):
            fwd[s][t] = sum(fwd[s_prime][t-1] * A[s_prime][s] * B[s][t]
                            for s_prime in xrange(N))
    prob_O = sum(fwd[s][T-1] * to_end[s] for s in xrange(N))
    print fwd
    return prob_O

In [747]:
fwd = forward()
print fwd

[[ 0.02479     0.          0.          0.        ]
 [ 0.          0.00005303  0.          0.        ]
 [ 0.          0.          0.00000184  0.        ]
 [ 0.          0.          0.          0.        ]]
9.2140914902e-12


##### Computing Backward Probability $\beta_t(j)$

**meaning:** The probability of seeing the observations from time $t+1$ to the end, given that having been in state$j$ at time $t$.

In [748]:
def backward(observation=O,HMM=LMD):
    Q, A, B = HMM['Q'], HMM['A'], HMM['B']
    from_start = HMM['from_start_probs']
    to_end = HMM['to_end_probs']
    O = observation
    N, T = len(Q), len(O)
    bwd = np.zeros((N,T))
    for s in xrange(N): # 0=START -> N-1=END.
        bwd[s][T-1] = to_end[s]
    for t in reversed(xrange(T-1)): 
        for s_prime in xrange(N): # s_prime for previous-state relative to s.
            bwd[s_prime][t] = sum(A[s_prime][s] * B[s][t+1] * bwd[s][t+1]
                                  for s in xrange(N))
    prob_O = sum(from_start[s]*B[s][0]*bwd[s][0] for s in xrange(N))
    print bwd
    return prob_O

In [749]:
bwd = backward()
print bwd

[[ 0.          0.          0.00000147  0.001     ]
 [ 0.          0.00000017  0.00000351  0.05      ]
 [ 0.          0.          0.00000501  0.001     ]
 [ 0.          0.00000008  0.00000647  0.13      ]]
9.2140914902e-12


##### Bigram HMM with FW-Algorithm (Single Observation)

**NB: Multiple-Observation FW HMM training requires Lagrange Multiplier Optimization, which digresses somewhat and is not necessarily in understanding the FW-algorithm as a special case of EM algorithms. **

In [816]:
import random
from collections import Counter

In [831]:
def random_generator(len_seq=100):
        # states = ['HOT','COLD']
        # observations = [1,2,3]
    rand = random.uniform(0,1)
    start_state = 'HOT' if rand > .5 else 'COLD'
    def transition(state):
        rand = random.uniform(0,1)
        if state=='HOT':  
            return 'HOT' if rand < .7 else 'COLD'
        else:
            return 'HOT' if rand < .4 else 'COLD'
        # transitions = np.array([[.7,.3],
        #                         [.4,.6]])
    def emission(state):
        rand = random.uniform(0,1)
        if state=='HOT':
            if rand >= .6: return 3
            elif rand < .2: return 1
            else: return 2
        else:
            if rand >= .9: return 3
            elif rand < .5: return 1
            else: return 2
        # emissions = np.array([[.2,.4,.4],
        #                       [.5,.4,.1]])
    tagged_sequence = []
    prev_state = start_state
    emitted = emission(prev_state)
    for i in xrange(len_seq):
        tagged_sequence.append((emitted,prev_state))
        prev_state = transition(prev_state)
        emitted = emission(prev_state)
    observation_sequence = [o for o,s in tagged_sequence]
    return tagged_sequence, observation_sequence
        

In [880]:
def fwd_bwd_hmm(Q, O, test_O, num_iters=100): # notation: cf. J&M ch6.5:12-21.
                                      # indexing for convenience in python: 1->N/T => 0->N-1/T-1.
    N, T = len(Q), len(O)
    # randomly initialize A (transitions) and B (emissions) matrices.
    A = np.random.rand(N,N)
    B = np.random.rand(N,T)
    
    from_start = to_end = 1/N # neutralize the effect from start->state & state->end.
    # forward-backward probabilities computation functions.
    #  notation: alpha_t(j)/beta_t(j) in J&M, but here we put rows for states,
    #            instead of having, e.g., alpha[t][s] for easier visualization.
    #            see previous forward/backward function to see why.
    def forward():        
        fwd = np.zeros((N,T))
        for s in xrange(N):
            fwd[s][0] = from_start * B[s][0]
        for t in xrange(1,T):
            for s in xrange(N):
                fwd[s][t] = sum(fwd[s_prime][t-1] * A[s_prime][s] * B[s][t]
                                for s_prime in xrange(N))
        return fwd
    def backward():
        bwd = np.zeros((N,T))
        for s in xrange(N): # 0=START -> N-1=END.
            bwd[s][T-1] = to_end
        for t in reversed(xrange(T-1)): 
            for s_prime in xrange(N): # s_prime for previous-state relative to s.
                bwd[s_prime][t] = sum(A[s_prime][s] * B[s][t+1] * bwd[s][t+1]
                                      for s in xrange(N))
        return bwd
    num_iter = 0
    while num_iter < num_iters:
        num_iter += 1
        if num_iter%100==0: print "... running the %dth EM iteration" % num_iter
        alpha = forward()
        beta = backward()
        prob_O = sum(alpha[s][T-1] for s in xrange(N)) * to_end 
            # alternatively: from_start * sum(B[s][0] * beta[s][0] for s in xrange(N))

        # E-step
        xi = np.zeros((T,N,N))
        gamma = np.zeros((T,N))
        for t in xrange(T-1):
            for i in xrange(N):
                for j in xrange(N):
                    not_quite_xi = alpha[i][t]*A[i][j]*B[j][t+1]
                    xi[t][i][j] = not_quite_xi / prob_O
        for t in xrange(T):
            for j in xrange(N):
                gamma[t][j] = (alpha[j][t]*beta[j][t]) / prob_O

        # M-step
        for i in xrange(N):
            for j in xrange(N):
                A[i][j] = sum(xi[t][i][j] for t in xrange(T-1)) / \
                          sum(xi[t][i][k] for t in xrange(T-1) for k in xrange(N))
                    # in the original algorithm, to_state is notated as j in
                    #  both the numerator and the denominator, which can be confusing.
        for j in xrange(N):
            for k in xrange(T):
                B[j][k] = sum(gamma[t][j]*B[j][k] for t in xrange(T)) / \
                          sum(gamma[t][j]*B[j][l] for t in xrange(T) for l in xrange(T))
                    # k: current emission; l: variable for all possible emissions.
    
    N, T = len(Q), len(test_O) # some repetition for readers' convenience.
    # viterbi
    viterbi = np.zeros((N,T))
    backpointer = np.zeros((N,T),dtype=int)
    for s in range(N):
        viterbi[s][0] = from_start * B[s][0]
        backpointer[s][0] = 0
    for t in xrange(1,T):
        for s in xrange(N):
            prev_viterbi = []
            prev_backpointer = []
            for s_prime in xrange(N):
                prev_times_transition = viterbi[s_prime][t-1] * A[s_prime][s]
                prev_viterbi.append(prev_times_transition * B[s][t])
                prev_backpointer.append(prev_times_transition)
            viterbi[s][t] = max(prev_viterbi)
            backpointer[s][t] = np.argmax(prev_backpointer)
    max_state = np.argmax(viterbi[:,-1])
    best_tagged_sequence = []
    for t in reversed(xrange(0,T)):
        best_tagged_sequence.insert(0,(test_O[t],Q[max_state]))
        max_state = backpointer[max_state][t]
    
    return A,B,best_tagged_sequence
    

##### DEMO

** Caution: w/o the extension to Lagrange Multiplier Optimization, unsupervised learning with EM will not work in general. This is especially true when the number of state gets large. **

In [861]:
tagged_train, observed_train = random_generator(100)
observed_test = observed_train[:10]

In [862]:
print tagged_train

[(2, 'HOT'), (1, 'COLD'), (1, 'COLD'), (2, 'COLD'), (1, 'COLD'), (2, 'COLD'), (1, 'COLD'), (2, 'COLD'), (2, 'HOT'), (2, 'COLD'), (2, 'COLD'), (3, 'COLD'), (2, 'HOT'), (2, 'COLD'), (3, 'HOT'), (2, 'COLD'), (2, 'COLD'), (2, 'COLD'), (2, 'COLD'), (3, 'HOT'), (2, 'HOT'), (2, 'HOT'), (1, 'HOT'), (2, 'HOT'), (2, 'HOT'), (1, 'COLD'), (2, 'HOT'), (3, 'HOT'), (1, 'COLD'), (3, 'HOT'), (2, 'HOT'), (3, 'HOT'), (2, 'HOT'), (1, 'HOT'), (1, 'COLD'), (1, 'COLD'), (1, 'COLD'), (2, 'COLD'), (2, 'COLD'), (1, 'COLD'), (2, 'COLD'), (1, 'HOT'), (2, 'HOT'), (2, 'HOT'), (2, 'HOT'), (3, 'HOT'), (2, 'HOT'), (2, 'HOT'), (3, 'HOT'), (3, 'HOT'), (3, 'HOT'), (3, 'HOT'), (2, 'HOT'), (1, 'HOT'), (3, 'HOT'), (3, 'HOT'), (3, 'HOT'), (3, 'COLD'), (1, 'COLD'), (1, 'HOT'), (3, 'HOT'), (3, 'HOT'), (2, 'COLD'), (3, 'COLD'), (2, 'COLD'), (3, 'HOT'), (2, 'HOT'), (2, 'COLD'), (1, 'HOT'), (2, 'HOT'), (2, 'HOT'), (2, 'COLD'), (3, 'HOT'), (2, 'COLD'), (3, 'HOT'), (3, 'HOT'), (2, 'HOT'), (3, 'HOT'), (1, 'COLD'), (3, 'HOT'), (2, 'H

In [863]:
print observed_train

[2, 1, 1, 2, 1, 2, 1, 2, 2, 2, 2, 3, 2, 2, 3, 2, 2, 2, 2, 3, 2, 2, 1, 2, 2, 1, 2, 3, 1, 3, 2, 3, 2, 1, 1, 1, 1, 2, 2, 1, 2, 1, 2, 2, 2, 3, 2, 2, 3, 3, 3, 3, 2, 1, 3, 3, 3, 3, 1, 1, 3, 3, 2, 3, 2, 3, 2, 2, 1, 2, 2, 2, 3, 2, 3, 3, 2, 3, 1, 3, 2, 2, 3, 2, 2, 2, 1, 3, 3, 3, 3, 1, 3, 2, 2, 1, 2, 3, 1, 1]


In [864]:
print observed_test

[2, 1, 1, 2, 1, 2, 1, 2, 2, 2]


In [865]:
%%time
A,B,predicted_sequence = fwd_bwd_hmm(['HOT','COLD'],observed_train,observed_test,num_iters=100)

... running the 10th EM iteration
... running the 20th EM iteration
... running the 30th EM iteration
... running the 40th EM iteration
... running the 50th EM iteration
... running the 60th EM iteration
... running the 70th EM iteration
... running the 80th EM iteration
... running the 90th EM iteration
... running the 100th EM iteration
CPU times: user 2min 14s, sys: 680 ms, total: 2min 15s
Wall time: 2min 15s


In [867]:
print predicted_sequence

[(2, 'HOT'), (1, 'COLD'), (1, 'COLD'), (2, 'COLD'), (1, 'COLD'), (2, 'COLD'), (1, 'COLD'), (2, 'COLD'), (2, 'COLD'), (2, 'COLD')]


In [868]:
print tagged_train[:10]

[(2, 'HOT'), (1, 'COLD'), (1, 'COLD'), (2, 'COLD'), (1, 'COLD'), (2, 'COLD'), (1, 'COLD'), (2, 'COLD'), (2, 'HOT'), (2, 'COLD')]


# EM Algorithm

** EM tutorial by Francisco Iacobelli; Code by Jacob Su Wang **

In [8]:
import random
from collections import Counter

In [10]:
def coin(theta=.5, num=100):
    return ['H' if random.uniform(0,1)<theta else 'T' for _ in xrange(num)]

In [11]:
biased_coin = coin(theta=.8)
fair_coin = coin(theta=.5)

In [15]:
print Counter(biased_coin)
print Counter(fair_coin)

Counter({'H': 84, 'T': 16})
Counter({'H': 52, 'T': 48})


In [ ]:
# TODO